# Interest-rate sensitivities via zero spread

In this notebook, I'll show a couple of different ways to calculate the sensitivity of an instrument price to changes in the interest-rate curve.

In [ ]:
import QuantLib as ql
today = ql.Date(8, ql.March, 2016)
ql.Settings.instance().evaluationDate = today

: 

In [ ]:
%matplotlib inline
from matplotlib.ticker import FuncFormatter
import numpy as np
import utils
from utils import format_rate

def plot_curves(*curves):
    fig, ax = utils.plot()
    ax.yaxis.set_major_formatter(FuncFormatter(lambda r,pos: format_rate(r)))
    ax.set_xlim(0,15)
    ax.set_xticks([0,5,10,15])
    times = np.linspace(0.0, 15.0, 400)
    for curve, style in curves:
        rates = [ curve.zeroRate(t, ql.Continuous).rate() for t in times ]
        ax.plot(times, rates, style)
def plot_curve(curve):
    plot_curves((curve,'-'))

#### Setup

Let's say we have an interest-rate curve, no matter how it was calculated. As an example, I'll use a curve bootstrapped over the 6-months deposit, a strip of 6-months FRAs and a number of swaps against the 6-months Euribor. All the market inputs are stored in quotes so that their values can be changed.

In [ ]:
quotes = [ ql.SimpleQuote(0.312/100) ]
helpers = [
    ql.DepositRateHelper(ql.QuoteHandle(quotes[0]),
                         ql.Period(6,ql.Months), 3,
                         ql.TARGET(), ql.Following,
                         False, ql.Actual360()) ]

for rate, months_to_start in [(0.293, 1), (0.272, 2), (0.260, 3),
                              (0.256, 4), (0.252, 5), (0.248, 6),
                              (0.254, 7), (0.261, 8), (0.267, 9),
                              (0.279, 10), (0.291, 11), (0.303, 12),
                              (0.318, 13), (0.335, 14), (0.352, 15),
                              (0.371, 16), (0.389, 17), (0.409, 18)]:
    quotes.append(ql.SimpleQuote(rate/100))
    helpers.append(ql.FraRateHelper(ql.QuoteHandle(quotes[-1]),
                                    months_to_start, ql.Euribor6M()))

for rate, tenor in [(0.424, 3), (0.576, 4), (0.762, 5), (0.954, 6),
                    (1.135, 7), (1.303, 8), (1.452, 9), (1.584, 10),
                    (1.809, 12), (2.037, 15), (2.187, 20), (2.234, 25),
                    (2.256, 30), (2.295, 35), (2.348, 40), (2.421, 50),
                    (2.463, 60)]:
    quotes.append(ql.SimpleQuote(rate/100))
    helpers.append(ql.SwapRateHelper(ql.QuoteHandle(quotes[-1]),
                                     ql.Period(tenor, ql.Years), ql.TARGET(),
                                     ql.Annual, ql.Unadjusted,
                                     ql.Thirty360(ql.Thirty360.BondBasis),
                                     ql.Euribor6M()))

rate_curve = ql.PiecewiseLogCubicDiscount(2, ql.TARGET(),
                                          helpers, ql.Actual365Fixed())
curve_handle = ql.RelinkableYieldTermStructureHandle(rate_curve)

Here's the curve, plotted over 15 years.

In [ ]:
plot_curve(rate_curve)

For illustration purposes, I'll be using the curve to price an interest-rate swap.  Let's create a 12-years swap starting in one month, with an annual schedule for the fixed leg and a semiannual schedule for the floating leg.  We'll use the curve above to forecast the floating-rate fixings, so we pass it to the `Euribor6M` instance that, in turn, we pass to the swap constructor.

In [ ]:
fixed_schedule = ql.Schedule(
    ql.Date(8, ql.April, 2016), ql.Date(8, ql.April, 2028),
    ql.Period(1, ql.Years), ql.TARGET(), ql.Following, ql.Following,
    ql.DateGeneration.Forward, False)
floating_schedule = ql.Schedule(
    ql.Date(8, ql.April, 2016), ql.Date(8, ql.April, 2028),
    ql.Period(6, ql.Months), ql.TARGET(), ql.Following, ql.Following,
    ql.DateGeneration.Forward, False)
index = ql.Euribor6M(curve_handle)
swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, 10000.0,
    fixed_schedule, 0.02, ql.Thirty360(ql.Thirty360.BondBasis),
    floating_schedule, index, 0.0, ql.Actual360()
)

Of course, we should use a different curve for discounting.  But let me just skip that part for brevity, and simply pass the same curve to the engine used by the swap.  The points I'm going to make won't suffer for this.

Once we've done this, we can finally get the value of the swap.

In [ ]:
swap.setPricingEngine(ql.DiscountingSwapEngine(curve_handle))
P0 = swap.NPV()
print(P0)

Now, let's say that this was you pricing a deal. And let's also say that you're interested (as you should) in how the swap price reacts to changes in the underlying rates.

#### Interest-rate sensitivities

If you're interested in the sensitivities of the price to the input rates, you can have them: shift any input rate by setting a perturbed value to the corresponding quote and recalculate the NPV.
For instance, you can bump the 6-months deposit rate by one basis point and get the new price as follows:

In [ ]:
bp = 1.0e-4
ref = quotes[0].value()
quotes[0].setValue(ref+1*bp)
print(swap.NPV())
quotes[0].setValue(ref)

(Also, don't forget to set the value back to the actual quoted rate when you're done).

This can be done for a single rate, as above, or for any number of rates; all of them, for instance...

In [ ]:
for q in quotes:
    q.setValue(q.value()+1*bp)
print(swap.NPV())
for q in quotes:
    q.setValue(q.value()-1*bp)

...so the above gives you the swap price when all the input rates move 1 basis point upwards; the difference between the new price and the old one will give you the DV01 of the swap. (Note that, depending on how you define it, you might want to shift either the forecast curve, the discount curve, or both.)

Different combinations of changes can also give you different stress scenarios; for instance, ones in which the curve tilts in some direction, or ones in which you only move the short end or the long end of the curve.  In doing so, though, you're constrained to use the nodes of the original curve. For instance, in the curve above there are no nodes between 20 and 25 years, thus there are no levers to pull in that interval.

As an alternative, you can take an approach in which you modify the curve as a whole independently of the underlying rates. For instance, to shift all the zero rates upwards, you can keep the original curve as it is and add one basis point to all its zero rates by means of the `ZeroSpreadedTermStructure` class.  To use it for pricing our swap, we'll store the original curve in a separate handle, add the spread, and link the new curve to the handle we're using for forecasting.  As usual, the swap price will react accordingly.

In [ ]:
base_curve = ql.YieldTermStructureHandle(rate_curve)
spread = ql.SimpleQuote(1*bp)

curve_handle.linkTo(
    ql.ZeroSpreadedTermStructure(base_curve, ql.QuoteHandle(spread)))
print(swap.NPV())

As we could expect, the result is close to what we got by shifting all the input rates (the difference is just one or two digits in the first decimal place) but not quite the same: modifying, say, an input swap rate doesn't have the same effect as applying the same change to the zero rates directly.  As usual, I'll trust you to know what you're doing in either case.

To get a more visual idea of what we're doing, we can also increase the spread and plot the resulting curve on top of the original one:

In [ ]:
spread.setValue(5*bp)
plot_curves((rate_curve,'-'), (curve_handle,'--'))

Another class, `SpreadedLinearZeroInterpolatedTermStructure` (for whose name I apologize on behalf of the library developers: I don't know what came over us) allows one to apply a spread which is interpolated linearly between a set of nodes, which of course are independent of the nodes of the underlying curve.

For instance, we can create a scenario in which we tilt the curve by taking equally spaced dates each year between now and 20 years, and define the corresponding spreads as negative in the short end, increasing until they reach zero at the 7-years mark, and more and more positive in the long end. Again, we can plot the resulting curve for comparison with the original one...

In [ ]:
spot = rate_curve.referenceDate()
dates = [ spot + ql.Period(n,ql.Years) for n in range(21)]
spreads = [ ql.QuoteHandle(ql.SimpleQuote((n-7)*bp))  for n in range(21) ]

curve_handle.linkTo(
    ql.SpreadedLinearZeroInterpolatedTermStructure(base_curve,
                                                   spreads, dates))
plot_curves((rate_curve,'-'), (curve_handle,'--'))

...and again, we can ask the swap for its price under this scenario.

In [ ]:
print(swap.NPV())

When using this technique, going back to the actual market quotes simply means linking the curve handle to the original curve:

In [ ]:
curve_handle.linkTo(rate_curve)
print(swap.NPV())

If you want more control of the shift (as asked, for instance, by user6142489 [on _Stack Overflow_](https://stackoverflow.com/questions/46279785/quantlib-building-key-rate-risks) who wanted to calculate key-rate risks) you might have to increase the number of nodes. If your nodes are one year apart as above, and if you modify, e.g., the node at 7 years, the interpolation scheme will cause the whole range between 6 and 8 years to change, and all coupons paid in that period to be affected.  The more nodes you have and the closer they are together, the more localized any change will be.